<a href="https://colab.research.google.com/github/cpk110/ChatGPT/blob/master/006_LangchainChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 여러문서에서 찾아서 답변하는 챗봇 만들기

랭체인(langchaining)이란?
- ChatGPT와 같은 대규모 언어 모델을 사용하여 어플리케이션을 쉽게 생성 할 수 있도록 설계 된 프레임워크이다.
- 랭체인이 도움이 되는 경우는 LLM에 외부의 '지식'이나 '계산능력'을 활용하게 하고 싶을 때이다.
- 자신이 학습한 것만으로 대화하던 LLM에게 '책'이나 '프로그램'을 전달해서 외부의 '지식'이나 '계산능력'을 활용 할 수 있게 하는 것이 랭체인의 역할이다.
- 예를 들어, 랭체인으로 LLM에 웹 검색 기능을 연결하면 LLM은 자신이 가진 지식만으로는 충분한 답변을 할 수 없는 경우에 웹 검색을 통해 최신 정보를 얻고 답변 할 수 있게 된다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/ai_chatbot_python/ChatGPT'

Mounted at /content/drive
/content/drive/MyDrive/ai_chatbot_python/ChatGPT


In [2]:
!pip install langchain openai==0.28.1 tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.3/205.3 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# 데이터 가져오기
# 003번과 동일한데이터 사용함

# !wget https://github.com/chatgpt-kr/chatgpt-api-tutorial/raw/main/ch05/data.zip
# !unzip data

### Setting up LangChain

OpenAI API Key
https://platfrom.openai.com/account/api-keys

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from collections import Counter

# 만약 import 오류가 날 경우 실행
# !pip install pydantic==<version_number>
# !pip install --upgrade langchain_core

In [5]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-'

# 동일한 방법으로 기존에는 아래 방법만 사용했었음
# import openai
# openai.api_key = 'sk-'

### Load multiple and process documents

In [6]:
# DirectoryLoader(path, glob)
# path : diretory 경로 / glob : 찾을 파일
loader = DirectoryLoader('./003/data', glob = '*.txt', loader_cls = TextLoader)

documents = loader.load()
print(type(documents))

print('문서의 갯수 : ', len(documents))

<class 'list'>
문서의 갯수 :  57


In [7]:
print('1번 문서 : ', documents[1])
print('-' * 20)
print('21번 문서 : ', documents[21])
print('-' * 20)

print(type(documents[1]))
print('-' * 20)
print(dir(documents[1]))

1번 문서 :  page_content='정책제목: 역세권 청년주택 공급\n\n서울시에서 청년과 신혼부부의 주거안정과 주거난 해소를 위해 대중교통이 편리한 역세권에 공공임대와 민간임대 주택을 시세보다 저렴하게 제공하는 정책을 진행하고 있습니다. 서울시청 전략주택공급과가 주관하며, 2016년 7월 1일부터 2026년 12월 31일까지 운영됩니다.\n\n이 정책은 다음과 같은 지원 내용을 포함하고 있습니다. 청년들이 부담 가능한 수준으로 주택에 입주할 수 있도록 임대보증금의 50% 지원(최대 4,500만원, 무이자), 임차 보증금의 무이자 지원, 청년과 신혼부부의 지원한도가 각각 4,500만원과 6,000만원입니다. 또한, 지원 대상은 공공지원민감임대주택(특별/일반공급) 입주 예정자입니다.\n\n지원 대상자는 대학생, 청년(19세~39세), 신혼부부(혼인합산 7년 이내) 중에서 차량 미운행자(차량 미소유자 포함)이며, 사업 대상 지역 거주민에게 우선적으로 공급됩니다. 장애인, 만 65세 이상 고령자, 임산부 또는 영유아(만 6세 미만)를 포함한 세대는 주차장 우선 제공 대상입니다.\n\n신청은 공공임대 주택은 SH서울주택도시공사 인터넷 청약 시스템을 통해, 민간임대 주택은 각 민간사업자의 홈페이지를 통해 접수됩니다. 신청절차에는 입주자 모집공고, 청약신청 접수, 서류심사 대상자 발표, 대상자 서류제출 및 소득, 자산조회, 소득자산 소명 등이 포함됩니다. 심사 및 발표 일정은 개별 모집 공고를 참조하시면 됩니다.\n\n자세한 사항은 서울시 주거정책과의 관련 사이트를 참고하시기 바랍니다.\n\n[참고사이트]\n서울시 주거정책 관련 사이트: http://housing.seoul.go.kr' metadata={'source': '003/data/32.txt'}
--------------------
21번 문서 :  page_content='정책이름: 서울희망 직업전문학교 장학금\n서울시 교육정책과에서 주관하는 교육 정책으로, 직업전문학교에서 경제적인 걱정 없이 학업을 수

### Split texts (텍스트 분할하기)

- ChatGPT API의 GPT-3.5-turbo 모델은 입력으로 사용하는 텍스트와 ChatGPT API가 반환하는 답변의 길이를 모두 포함하여 최대 4,096 토큰을 처리할 수 있다.
- 예를 들어, ChatGPT API 입력으로 약 3,000토큰 정도의 길이를 가진 입력을 사용하면 ChatGPT는 최대 약 1,000토큰의 답변을 할 수 있는 구조이다.
- 입력의 길이가 4,096의 길이에 인접한 입력을 사용하면 ChatGPT API가 답변을 하던 도중 4,096개의 토큰을 모두 소진하면 더 이상 답변을 생성할 수 없어 텍스트가 중간에 끊길 수 있다. 또한 입력의 길이가 4,096을 넘으면 ChatGPT의 입력으로 사용할 수 없다는 에러가 발생한다.
- 이런 현상 때문에 너무 긴 문서들은 한 번에 입력으로 사용할 수 없으므로 여러 개의 뭉치(chunk)로 자른 후 ChatGPT API를 여러 번 호출하여 처리해야 한다.
- 랭체인의 RecursiveCharacterTextSplitter는 이를 위해 사용하는 도구로, 주어진 문서를 더 작은 단위의 텍스트 뭉치(chunk)로 잘라주는 역할을 한다.
-chunk_size에는 텍스트의 분할 기준이 되는 길이를 설정하고, chunk_overlap에는 텍스트를 분할할 때 텍스트의 내용을 얼마나 겹쳐서 자를 것인지 지정한다.

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
texts = text_splitter.split_documents(documents)

print('분할 된 텍스트의 갯수 : ', len(texts))

분할 된 텍스트의 갯수 :  64


In [9]:
texts[0]

Document(page_content='정책제목: 일생활균형 서울형 강소기업 지원\n서울시는 청년과 중소기업의 일자리 미스매치 문제 해소를 위해 서울형 강소기업 발굴 및 지원을 통해 청년에게는 우수 중소기업에서 일할 기회를, 중소기업에게는 청년 인재를 채용할 기회를 제공하는 일자리 정책을 추진하고 있습니다. 이 정책은 2023년 1월 1일부터 2023년 12월 31일까지 사업운영이 이루어지며, 사업 신청은 동일한 기간 동안 가능합니다. 지원 내용으로는 근무 환경 개선금, 육아휴직 대체 청년 인턴 인건비 지원 등이 포함되어 있습니다. 추가로 50개 기업이 추가 선정될 예정입니다. 신청자격은 서울시 소재 중소기업 중 공공기관 인증을 받은 기업으로 제한되며, 기업의 재무제표 확인을 위해 2년의 최소 업력이 필요합니다. 신청은 해당 사이트(http://www.seouljobnow.co.kr/)에서 온라인으로 진행하며, 제출해야 할 서류는 공고를 참조하시기 바랍니다. 심사 및 발표는 일자리 창출 실적, 근로자 처우 수준, 기업 우수성, 고용 안정성 및 임금 수준, 복지제도 및 일생활 균형제도 운영 등을 기준으로 이루어집니다. 운영기관은 서울시청 일자리정책과입니다. 자세한 내용은 http://www.seouljobnow.co.kr/에서 확인하실 수 있습니다.', metadata={'source': '003/data/46.txt'})

In [10]:
source_lst  = []
for i in range(0, len(texts)):
  source_lst.append(texts[i].metadata['source'])

element_counts = Counter(source_lst)
print(element_counts, '\n')

filtered_counts = {key : value for key, value in element_counts.items() if value >= 2}
print('2개 이상으로 분할 된 문서 : ', filtered_counts)
print('분할 된 텍스트의 갯수 : ', len(documents) + len(filtered_counts))

Counter({'003/data/40.txt': 2, '003/data/22.txt': 2, '003/data/31.txt': 2, '003/data/49.txt': 2, '003/data/23.txt': 2, '003/data/48.txt': 2, '003/data/36.txt': 2, '003/data/46.txt': 1, '003/data/32.txt': 1, '003/data/1.txt': 1, '003/data/45.txt': 1, '003/data/56.txt': 1, '003/data/52.txt': 1, '003/data/12.txt': 1, '003/data/17.txt': 1, '003/data/34.txt': 1, '003/data/27.txt': 1, '003/data/3.txt': 1, '003/data/35.txt': 1, '003/data/50.txt': 1, '003/data/16.txt': 1, '003/data/20.txt': 1, '003/data/26.txt': 1, '003/data/7.txt': 1, '003/data/55.txt': 1, '003/data/51.txt': 1, '003/data/24.txt': 1, '003/data/18.txt': 1, '003/data/30.txt': 1, '003/data/25.txt': 1, '003/data/39.txt': 1, '003/data/29.txt': 1, '003/data/42.txt': 1, '003/data/33.txt': 1, '003/data/19.txt': 1, '003/data/2.txt': 1, '003/data/37.txt': 1, '003/data/28.txt': 1, '003/data/11.txt': 1, '003/data/53.txt': 1, '003/data/47.txt': 1, '003/data/41.txt': 1, '003/data/6.txt': 1, '003/data/54.txt': 1, '003/data/57.txt': 1, '003/d

### Create Chroma DB

- 그 동안에는 OpenAI Embedding API를 이용해 텍스트를 임베딩하고, 코사인 유사도를 계산해 유사한 텍스트를 가져오는 작업을 했다.

  ```python
    from openai.embeddings_utils import get_embedding
    get_embedding('저는 배가 고파요', engine = 'text-embedding-ada-002')
    
    def cos_sim(A, B):
      return dot(A, B) / (norm(A) * norm(B))
  ```

- ChromaDB는 이러한 과정들을 기능 별로 구현하여 사용자가 벡터를 좀 더 쉽게 다룰 수 있게 도와주는 편리한 벡터 응용 도구이다.
- Chroma.from_documents( )함수를 이용해 벡터 도구 객체를 선언한다.  
이때 documents에는 벡터화의 단위가 될 텍스트 리스트를 매개변수로 전달하고, embedding에는 어떤 종류의 임베딩을 사용 할 것인지를 기재한다.

In [11]:
# embedding은 문자를 벡터화하는 과정
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents = texts, embedding = embedding)
print(vectordb)
print(dir(vectordb))

['_Chroma__query_collection', '_LANGCHAIN_DEFAULT_COLLECTION_NAME', '__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_asimilarity_search_with_relevance_scores', '_client', '_client_settings', '_collection', '_cosine_relevance_score_fn', '_embedding_function', '_euclidean_relevance_score_fn', '_get_retriever_tags', '_max_inner_product_relevance_score_fn', '_persist_directory', '_select_relevance_score_fn', '_similarity_search_with_relevance_scores', 'aadd_documents', 'aadd_texts', 'add_documents', 'add_images', 'add_texts', 'adelete', 'afrom_documents', 'afrom_texts', 'amax_marginal_relevance_search', 'amax_marginal_relevance_sea

### Make a retriever (검색기)

- 벡터 도구 객체를 선언하고 나면 as_retriever( ) 함수를 이용해 입력 된 텍스트로 부터 유사한 텍스트를 찾아주는 검색기(retriever, 리트리버)를 선언한다.

In [12]:
retriever = vectordb.as_retriever()
print(type(retriever))

<class 'langchain_core.vectorstores.VectorStoreRetriever'>


- get_relevant_documents('입력데이터')를 실행하면  
'입력데이터'와 유사한 관계에 있는 텍스트를 리턴한다.

In [13]:
docs = retriever.get_relevant_documents('신혼 부부를 위한 정책이 있어?')
print('유사 문서 갯수 : ', len(docs))
print('-' * 20)
print('첫번째 유사 문서 : ', docs[0])
print('-' * 20)
print('각 유사 문서의 출처 : ')
for doc in docs:
  print(doc.metadata['source'])

print('-' * 20)
docs[0]

유사 문서 갯수 :  4
--------------------
첫번째 유사 문서 :  page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.' metadata={'source': '003/data/23.txt'}
--------------------
각 유사 문서의 출처 : 
003/data/23.txt
003/data/23.txt
003/data/40.txt
003/data/39.txt
--------------------


Document(page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.', metadata={'source': '003/data/23.txt'})

### 결과를 k개 반환

In [14]:
# search_kwargs = {'k' : 문서의 갯수}
# 유사도 문서 갯수가 지정한 갯수만큼 리턴해준다.
retriever = vectordb.as_retriever(search_kwargs = {'k' : 2})

In [15]:
docs = retriever.get_relevant_documents('신혼 부부를 위한 정책이 있어?')

for doc in docs:
  print(doc.metadata['source'])

003/data/23.txt
003/data/23.txt


### Make a chain

```
Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{텍스트}

{질문}
```

In [16]:
# RetrievalQA.from_chain_type( ) 함수의 llm 매개변수 값으로 ChatOpenAI( )를 지정하고,
# 내부매개변수로 model_name의 값으로 'gpt-3.5-turbo'를 지정하면 ChatGPT API를 사용 할 수 있다.

# temperature = 0 이면 같은 답변을 반환하고, 1이면 랜덤이 발생한다. (default = 1)
# chain_type = 'stuff'이면 내부적으로 위에 있는 문장처럼 '{텍스트} {질문}'을 수행한다.
#               {텍스트}에는 입력데이터와 유사도가 높은 텍스트의 본문이 삽입되고,
#               {질문}에는 입력텍스트가 삽입되는 구조이다.
#               그동안 사용자가 직접 프롬프트를 작성한 것과 달리 여기서는 이미 작성 된 프롬프트를 사용한다.

qa_chain = RetrievalQA.from_chain_type(
    llm = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature = 0),
    chain_type = 'stuff',
    retriever = retriever,
    return_source_documents = True
)

### Query

In [17]:
input_text = '대출과 관련 된 정책이 궁금합니다.'
chatbot_response = qa_chain(input_text)

print(chatbot_response)
print(dir(chatbot_response))
print(chatbot_response.keys())

{'query': '대출과 관련 된 정책이 궁금합니다.', 'result': "어떤 종류의 대출 정책에 대해 궁금하신가요? 서울시에서는 주거마련을 위한 대출 지원 정책인 '역세권청년주택 주거비지원' 정책을 운영하고 있습니다. 이 정책은 청년과 신혼부부의 주거비 부담을 경감하기 위해 임차보증금 대출을 지원하는 것으로, 임차보증금에 대해 무이자 지원을 제공합니다. 이 외에도 다양한 대출 정책이 있을 수 있으니, 구체적으로 어떤 대출에 대해 알고 싶으신지 자세히 알려주시면 도움을 드릴 수 있을 것입니다.", 'source_documents': [Document(page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.', metadata={'source': '003/data/23.txt'}), Document(page_c

In [21]:
def get_chatbot_response(chatbot_response):
  print(chatbot_response['result'].strip())
  print('\n문서출처 : ')
  for source in chatbot_response['source_documents']:
    print(source.metadata['source'])

- '신혼 부부의 신혼집 마련을 위한 정책이 있을까?'라는  
임의의 입력에 대해서 챗봇의 답변을 확인

In [20]:
input_text = '신혼 부부의 신혼집 마련을 위한 정책이 있을까?'
chatbot_response = qa_chain(input_text)
get_chatbot_response(chatbot_response)

네, 서울시에서는 신혼부부의 주거마련을 위한 정책을 운영하고 있습니다. 이 정책은 신혼부부의 임차보증금을 지원하는 내용으로, 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다. 자세한 내용은 서울주거포털(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.

문서출처 : 
003/data/23.txt
003/data/23.txt


In [22]:
# print(chatbot_response)
chatbot_response

{'query': '신혼 부부의 신홉집 마련을 위한 정책이 있을까?',
 'result': '네, 서울시에서는 신혼부부의 주거마련을 위한 정책을 운영하고 있습니다. 이 정책은 신혼부부의 임차보증금을 지원하는 내용으로, 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다. 자세한 내용은 서울주거포털(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.',
 'source_documents': [Document(page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.', metadata={'source': '003/data/23.txt'}),
  Document(page_content='정책제목: 신혼부부 임차보증금 지원\n정부에서

In [ ]:
input_text = '전세자금이 부족한 사람을 위한 정책 이름이 뭐야?'
llm_response = qa_chain(input_text)
get_chatbot_response(llm_response)

In [ ]:
query = '희망 두배 청년 통장은 어떤걸 지원하니?'
# query = '희망 두배 청년 통장의 지원 조건은?'
# query = '희망 두배 청년 통장의 마감 기한은?'
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

### Gradio로 챗봇의 UI만들기

In [23]:
# 4.xx 버전은 테스트 자료가 부족하여 3.40.1로 진행
!pip install gradio==3.40.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 5.4 MB/s eta 0:00:

- To create a public link, set share=True in launch().  
Running on https://localhost:7860/  
문구가 나오면 위의 localhost로 시작하는 주소 클릭

In [26]:
import gradio as gr

# 인터페이스를 생성
with gr.Blocks() as demo:
  chatbot = gr.Chatbot(label = '청년정책챗봇')  # '청년정책챗봇' 레이블을 좌측 상단에 구성
  msg = gr.Textbox(label = '질문해주세요!')  # 하단의 채팅창 레이블
  clear = gr.Button('대화 초기화')  # 초기화 버튼

  # 챗봇의 답변을 처리하는 함수
  def respond(message, chat_history):
    result = qa_chain(message)
    bot_message = result['result']
    bot_message += ' # source : '

    # 답변의 출처를 표기
    for i, doc in enumerate(result['source_documents']):
      bot_message += '[' + str(i + 1) + ']' + doc.metadata['source'] + ' '

    # 채팅기록에 사용자의 메시지와 봇의 응답을 추가
    chat_history.append((message, bot_message))
    return '', chat_history

  # 사용자의 입력을 제출(submit)하면 respond 함수가 호출
  msg.submit(respond, [msg, chatbot], [msg, chatbot])

  # '초기화' 버튼을 클릭하면 채팅기록을 초기화
  clear.click(lambda : None, None, chatbot, queue = False)

# 인터페이스 실행
demo.launch(debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
